# to_list - Flexible List Conversion Utility

`to_list()` converts arbitrary inputs to lists with powerful transformations:

**Core Features:**
- **Smart Type Handling**: Strings, enums, mappings, models, iterables
- **Recursive Flattening**: Nested iterables → flat list
- **Deduplication**: Two-strategy unique value extraction (direct + hash-based)
- **Null Filtering**: Remove None and undefined values
- **Value Extraction**: Get values from enums/mappings instead of keys
- **Tuple/Set Control**: Include or exclude from flattening

In [1]:
from enum import Enum

from lionherd_core.ln import to_list
from lionherd_core.ln._to_list import ToListParams

## 1. Basic Conversion

Handles common types intelligently - strings treated as single items, iterables unpacked.

In [2]:
# Single values
print(f"int: {to_list(42)}")
print(f"str: {to_list('hello')}")
print(f"None: {to_list(None)}")

# Iterables
print(f"list: {to_list([1, 2, 3])}")
print(f"tuple: {to_list((4, 5, 6))}")
print(f"set: {to_list({7, 8, 9})}")

int: [42]
str: ['hello']
None: []
list: [1, 2, 3]
tuple: [4, 5, 6]
set: [8, 9, 7]


## 2. Flatten Mode - Nested Structures

Recursively flattens nested iterables into a single flat list.

In [3]:
nested = [1, [2, [3, [4, 5]], 6], 7]

# Without flatten - preserves structure
print(f"No flatten: {to_list(nested, flatten=False)}")

# With flatten - fully flattened
print(f"Flatten: {to_list(nested, flatten=True)}")

No flatten: [1, [2, [3, [4, 5]], 6], 7]
Flatten: [1, 2, 3, 4, 5, 6, 7]


In [4]:
# Complex nesting with mixed types
complex_nested = [1, [2, 3], [[4, 5], [6, [7, 8]]], 9]
flat = to_list(complex_nested, flatten=True)
print(f"Complex flatten: {flat}")
print(f"Length: {len(flat)} (all elements at top level)")

Complex flatten: [1, 2, 3, 4, 5, 6, 7, 8, 9]
Length: 9 (all elements at top level)


## 3. Dropna Mode - Filter Null Values

Removes None and sentinel values (Undefined/Unset) from the output.

In [5]:
from lionherd_core.types import Undefined, Unset

data_with_nulls = [1, None, 2, Undefined, 3, Unset, 4]

# Keep nulls
print(f"With nulls: {to_list(data_with_nulls, dropna=False)}")

# Drop nulls
print(f"Drop nulls: {to_list(data_with_nulls, dropna=True)}")

With nulls: [1, None, 2, Undefined, 3, Unset, 4]
Drop nulls: [1, 2, 3, 4]


In [6]:
# Combined with flatten
nested_with_nulls = [1, [None, 2], [3, [None, 4]], None]
clean_flat = to_list(nested_with_nulls, flatten=True, dropna=True)
print(f"Flatten + dropna: {clean_flat}")

Flatten + dropna: [1, 2, 3, 4]


## 4. Unique Mode - Deduplication

Removes duplicates while preserving order. Requires `flatten=True`.

In [7]:
duplicates = [1, 2, 3, 2, 4, 1, 5, 3]

# Without unique
print(f"With duplicates: {to_list(duplicates, flatten=True)}")

# With unique - order preserved
print(f"Unique: {to_list(duplicates, flatten=True, unique=True)}")

With duplicates: [1, 2, 3, 2, 4, 1, 5, 3]
Unique: [1, 2, 3, 4, 5]


In [8]:
# Unique requires flatten
try:
    to_list([1, 2, 3], unique=True, flatten=False)
except ValueError as e:
    print(f"✓ Error caught: {e}")

✓ Error caught: unique=True requires flatten=True


## 5. Deduplication Strategies

Two-strategy approach handles both hashable and unhashable types:
1. **Direct**: Fast path for hashable items (int, str, tuple)
2. **Hash-based fallback**: For unhashable items (dict, list, BaseModel)

In [9]:
# Hashable types - direct strategy
hashable = [1, "a", (2, 3), 1, "a", (2, 3)]
unique_hashable = to_list(hashable, flatten=True, unique=True)
print(f"Hashable dedup: {unique_hashable}")

Hashable dedup: [1, 'a', (2, 3)]


In [10]:
# Unhashable types - hash-based fallback
dicts = [{"a": 1}, {"b": 2}, {"a": 1}, {"c": 3}]
unique_dicts = to_list(dicts, flatten=True, unique=True)
print(f"Dict dedup: {unique_dicts}")
print(f"Length: {len(unique_dicts)} (duplicate removed)")

Dict dedup: [{'a': 1}, {'b': 2}, {'c': 3}]
Length: 3 (duplicate removed)


In [11]:
# Mixed hashable and unhashable - triggers fallback
mixed = [1, {"a": 1}, 2, {"a": 1}, 1, {"b": 2}]
unique_mixed = to_list(mixed, flatten=True, unique=True)
print(f"Mixed dedup: {unique_mixed}")
print(f"Deduplicated both types: {len(unique_mixed)} items")

Mixed dedup: [1, {'a': 1}, 2, {'b': 2}]
Deduplicated both types: 4 items


## 6. use_values Mode - Extract from Enums/Mappings

Extract values instead of keys from enums and mappings.

In [12]:
# Enum handling
class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3


# Get enum members
print(f"Enum members: {to_list(Color, use_values=False)}")

# Get enum values
print(f"Enum values: {to_list(Color, use_values=True)}")

Enum members: [<Color.RED: 1>, <Color.GREEN: 2>, <Color.BLUE: 3>]
Enum values: [1, 2, 3]


In [13]:
# Mapping handling
mapping = {"a": 1, "b": 2, "c": 3}

# Keep as single item
print(f"Mapping as item: {to_list(mapping, use_values=False)}")

# Extract values
print(f"Mapping values: {to_list(mapping, use_values=True)}")

Mapping as item: [{'a': 1, 'b': 2, 'c': 3}]
Mapping values: [1, 2, 3]


In [14]:
# String handling with use_values
text = "hello"

# String as single item (default)
print(f"String as item: {to_list(text, use_values=False)}")

# String as char list
print(f"String as chars: {to_list(text, use_values=True)}")

String as item: ['hello']
String as chars: ['h', 'e', 'l', 'l', 'o']


## 7. flatten_tuple_set Mode - Control Tuple/Set Flattening

By default, tuples and sets are NOT flattened (treated as atomic). Enable to flatten them.

In [15]:
data_with_tuples = [1, (2, 3), [4, (5, 6)], {7, 8}]

# Default - tuples/sets preserved
print(f"Tuples preserved: {to_list(data_with_tuples, flatten=True, flatten_tuple_set=False)}")

# Flatten tuples/sets too
print(f"Tuples flattened: {to_list(data_with_tuples, flatten=True, flatten_tuple_set=True)}")

Tuples preserved: [1, (2, 3), 4, (5, 6), {8, 7}]
Tuples flattened: [1, 2, 3, 4, 5, 6, 8, 7]


In [16]:
# Nested tuples
nested_tuples = [1, (2, (3, 4)), [(5, 6), 7]]

result_preserved = to_list(nested_tuples, flatten=True, flatten_tuple_set=False)
result_flattened = to_list(nested_tuples, flatten=True, flatten_tuple_set=True)

print(f"Preserved: {result_preserved}")
print(f"Flattened: {result_flattened}")

Preserved: [1, (2, (3, 4)), (5, 6), 7]
Flattened: [1, 2, 3, 4, 5, 6, 7]


## 8. Combined Transformations

Chain multiple options for powerful data processing.

In [17]:
# Realistic data cleaning scenario
messy_data = [
    1,
    None,
    [2, 3, None],
    [[4, 5], [5, 6]],
    None,
    7,
    [8, 4, 1],  # duplicates
]

# Flatten + dropna + unique
cleaned = to_list(messy_data, flatten=True, dropna=True, unique=True)
print(f"Messy: {messy_data}")
print(f"Cleaned: {cleaned}")
print("✓ Flattened, null-free, deduplicated")

Messy: [1, None, [2, 3, None], [[4, 5], [5, 6]], None, 7, [8, 4, 1]]
Cleaned: [1, 2, 3, 4, 5, 6, 7, 8]
✓ Flattened, null-free, deduplicated


In [18]:
# Extract and deduplicate enum values
class Status(Enum):
    PENDING = "pending"
    ACTIVE = "active"
    DONE = "done"


# Multiple status lists with duplicates
status_data = [
    [Status.PENDING, Status.ACTIVE],
    [Status.ACTIVE, Status.DONE],
    None,
]

# Get unique status values
unique_values = to_list(
    status_data,
    flatten=True,
    dropna=True,
    unique=True,
    use_values=True,
)
print(f"Unique status values: {unique_values}")

Unique status values: [<Status.PENDING: 'pending'>, <Status.ACTIVE: 'active'>, <Status.DONE: 'done'>]


## 9. ToListParams - Reusable Configurations

Store common parameter combinations for reuse.

In [19]:
# Create reusable configs
clean_params = ToListParams(flatten=True, dropna=True, unique=True)
extract_params = ToListParams(use_values=True, flatten=False)

# Apply to data
messy = [1, None, [2, 1, None], 3, 2]
cleaned = clean_params(messy)
print(f"Clean config result: {cleaned}")

Clean config result: [1, 2, 3]


In [20]:
# Override params at call time
override_result = clean_params(messy, unique=False)
print(f"With unique override: {override_result}")
print(f"Original config unchanged: {clean_params.unique}")

With unique override: [1, 2, 1, 3, 2]
Original config unchanged: True


In [21]:
# Enum extraction config
mapping = {"a": 1, "b": 2}
extracted = extract_params(mapping)
print(f"Extract config result: {extracted}")

Extract config result: [1, 2]


## 10. Edge Cases and Type Handling

Special handling for BaseModel, bytes, and complex nested structures.

In [22]:
from pydantic import BaseModel


class Person(BaseModel):
    name: str
    age: int


# BaseModel treated as atomic (not iterable)
person = Person(name="Alice", age=30)
result = to_list([person, [person]], flatten=True)
print(f"BaseModel handling: {result}")
print(f"Type preserved: {type(result[0]).__name__}")

BaseModel handling: [Person(name='Alice', age=30), Person(name='Alice', age=30)]
Type preserved: Person


In [23]:
# Bytes handling
byte_data = b"hello"

# Bytes as single item (default)
print(f"Bytes as item: {to_list(byte_data, use_values=False)}")

# Bytes as byte list
print(f"Bytes as list: {to_list(byte_data, use_values=True)}")

Bytes as item: [b'hello']
Bytes as list: [104, 101, 108, 108, 111]


In [24]:
# Deeply nested with mixed types
complex_data = [
    1,
    [2, (3, 4)],
    [[5, {6, 7}], None],
    {"a": [8, 9]},
    "string",
]

result = to_list(
    complex_data,
    flatten=True,
    dropna=True,
    flatten_tuple_set=False,
)
print(f"Complex flatten: {result}")
print("Tuples/sets/dicts preserved as atoms: ✓")

Complex flatten: [1, 2, (3, 4), 5, {6, 7}, {'a': [8, 9]}, 'string']
Tuples/sets/dicts preserved as atoms: ✓


## Summary Checklist

**Core Capabilities:**
- ✅ Smart type handling (str, enum, mapping, model, iterable)
- ✅ Recursive flattening of nested structures
- ✅ Two-strategy deduplication (direct + hash-based)
- ✅ Null value filtering (None/Undefined/Unset)
- ✅ Value extraction from enums and mappings
- ✅ Tuple/set flattening control
- ✅ Parameter combinations via ToListParams

**Key Behaviors:**
- Strings/bytes treated as atomic by default (use `use_values=True` to split)
- Tuples/sets preserved by default (use `flatten_tuple_set=True` to flatten)
- BaseModel instances always atomic (never flattened)
- Mappings atomic unless `use_values=True`
- Order preserved during deduplication
- `unique=True` requires `flatten=True`

**Use Cases:**
- Data cleaning pipelines
- Config normalization
- Enum value extraction
- Nested list flattening
- Deduplication with complex types